#머신러닝으로 예측하기

-모델: 머신러닝으로 학습된 패턴을 저장하는 소프트웨어 객체 \
-지도학습: 데이터에 있는 각 샘플에 대한 타깃을 알고 있는 경우 \
-입력: 타깃을 맞추기 위해 모델이 재료로 사용하는 데이터 \
-비지도 학습: 입력데이터만 있는 경우 \
-딥러닝: 신경망 알고리즘을 사용하는 머신러닝 알고리즘 (지도, 비지도학습에 모두 사용 가능)

**훈련세트와 테스트세트로 나누기**

In [2]:
import gdown
gdown.download('https://bit.ly/3pK7iuu', 'ns_book7.csv', quiet=False)

import pandas as pd

ns_book7 = pd.read_csv('ns_book7.csv', low_memory=False)
ns_book7.head()

Downloading...
From: https://bit.ly/3pK7iuu
To: /content/ns_book7.csv
100%|██████████| 53.8M/53.8M [00:01<00:00, 41.2MB/s]


,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,인공지능과 흙,김동훈 지음,민음사,2021,9788937444319,NaN,NaN,NaN,NaN,1,0,2021-03-19
1,2,가짜 행복 권하는 사회,김태형 지음,갈매나무,2021,9791190123969,NaN,NaN,NaN,NaN,1,0,2021-03-19
2,3,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,블랙피쉬,2021,9788968332982,NaN,NaN,NaN,NaN,1,0,2021-03-19
3,4,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",문학세계사,2021,9788970759906,NaN,NaN,NaN,NaN,1,0,2021-03-19
4,5,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,김영사,2021,9788934990833,NaN,NaN,NaN,NaN,1,0,2021-03-19


In [5]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(ns_book7, random_state=42)
#분할 비율 조정시, test_size 매겨변수에 0~1

In [7]:
print(len(train_set), len(test_set))  #75%, 25%

282577 94193


**선형 회귀 모델 훈련하기**

In [8]:
X_train = train_set[['도서권수']]
y_train = train_set['대출건수']

print(X_train.shape, y_train.shape)

#사이킷런은 입력으로 2차원 배열, 타깃으로 1차원 배열을 기대.

(282577, 1) (282577,)


In [14]:
X_train  #도서권수는 특성

,도서권수
351805,1
150275,1
97647,1
367482,1
93337,1
...,...
259178,1
365838,1
131932,1
146867,1


In [13]:
y_train  #1차원 시리즈 객체

351805     0
150275     1
97647     11
367482     0
93337      4
          ..
259178     0
365838     0
131932    21
146867     4
121958    36
Name: 대출건수, Length: 282577, dtype: int64

In [9]:
# 선형회귀 모델에 두가지 데이터를 입력하여 훈련
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

**훈련된 모델을 평가하기: 결정계수**

In [10]:
X_test = test_set[['도서권수']]
y_test = test_set['대출건수']

lr.score(X_test, y_test)

#0~1 값 반환
#약 0.1로 두세트 간의 관련성이 적음.


0.10025676249337112

-결정계수: (R^2) = 1 - {(타깃-예측)^2 / (타깃-타깃평균)^2} \
-예측이 평균에 가까워질수록 결정계수는 1에 가까워짐

In [15]:
#대출건수로 대출건수 예측하는 모델
lr.fit(y_train.to_frame(), y_train)
lr.score(y_test.to_frame(), y_test)

1.0

-fit함수 호출시 선형회귀 모델은 기울기(coef), 절편(intercept) 속성을 학습함

In [16]:
print(lr.coef_, lr.intercept_)
# y=x

[1.] -3.907985046680551e-14


**카테고리 예측하기: 로지스틱 회귀**

-회귀: 타깃이 실수인 문제 \
-분류: 타깃이 카테고리(이진, 다중) \
-이진분류: 음성클래스(0), 양성클래스(1)

**로지스틱 회귀모델 훈련하기**

In [17]:
borrow_mean = ns_book7['대출건수'].mean()
y_train_c = y_train > borrow_mean
y_test_c = y_test > borrow_mean

In [18]:
from sklearn.linear_model import LogisticRegression

logr = LogisticRegression()
logr.fit(X_train, y_train_c)
logr.score(X_test, y_test_c)
#정확도: 입력데이터 중 정답을 맞힌 비율

0.7106154385145393

In [19]:
y_test_c.value_counts()
# 비율이 불균형 -> 모델이 모두 False라고 예측했어도 69%는 나옴. 따라서 위의 71%는 높은 수치가 아님.

False    65337
True     28856
Name: 대출건수, dtype: int64

In [20]:
from sklearn.dummy import DummyClassifier

dc = DummyClassifier()
dc.fit(X_train, y_train_c)
dc.score(X_test, y_test_c)
#분류의 더미일 경우 가장 많은 클래스를 예측으로 출력
#아래값이 모델의 기준점이 되며, 이보다 낫지 않은 값을 출력시 좋은 모델이라고 말할 수 없다.

0.6936502712515792

In [21]:
lr.fit(X_train, y_train)

LinearRegression()

In [22]:
y_pred = lr.predict(X_test)

In [24]:
#평균 절대값 오차로 모델 평가하기
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, y_pred)
#예측이 10정도 차이 -> 좋은 모델이 아님.


10.35809175285386